<h1>Introduction</h1>
Clone all files, install all libraries

In [ ]:
pip install datasets torch torchaudio jiwer transformers openai-whisper faster-whisper soundfile librosa


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 11.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 

In [2]:
!git clone https://github.com/kchase9/creolese-audio-dataset.git

Cloning into 'creolese-audio-dataset'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 90 (delta 8), reused 15 (delta 5), pack-reused 70 (from 2)
Receiving objects: 100% (90/90), 455.70 MiB | 31.36 MiB/s, done.
Resolving deltas: 100% (8/8), done.
Updating files: 100% (71/71), done.


<h1>Get all global variables and libraries added</h1>


In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print(device)
print(torch_dtype)

cpu
torch.float32


In [27]:
audio_path = "/content/creolese-audio-dataset/Audio Files"
transcription_path = "/content/creolese-audio-dataset/transcripts.json"

# Add the pretrained Whisper model

In [23]:
whisper_model_id = "openai/whisper-large-v3"
whisper_processor = AutoProcessor.from_pretrained(whisper_model_id)

In [7]:
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    whisper_model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True
).to(device)

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

In [8]:
from transformers import pipeline

asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=whisper_processor.tokenizer,
    feature_extractor=whisper_processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)


Device set to use cpu


## Display the preliminary results

In [29]:
result = asr_pipeline("/content/drive/MyDrive/Colab Notebooks/FYP Research Artefacts/proof of concept/3.wav", return_timestamps=True)
print(result["text"])

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


 I never really ask she about she situation because when I dealing with a girl, I don't care about she past. I don't really care about down though, you know. When I dealing with a girl, it means she. Nobody can know, nobody blies to know. You know, we could keep it low key, you know. This is me fam. But, me know this man is behaving so that I tell you just so. Man tekse mi 12 krak in yi nèk 3 krak in yi mè anèk Mè wò If i wan hi sen peti wèk hi an shi E tmane is do dwen nan Fò dis lèl man bè Twil tak I don tèl yi nan nan I jès lèf tèz dò


# Run pretrained evaluations

## Import libraries and load files

In [11]:
pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 79.6 MB/s eta 0:00:00


In [16]:
import os
import json
import jiwer

with open(transcription_path, "r") as f:
    dataset = json.load(f)

## Officially run the evaluations
### Make and store the predictions

In [ ]:
ground_truth = []
predictions= []

for item in dataset:
  filename= ""
  audio_path = "/content/creolese-audio-dataset/Audio Files"
  filename = os.path.basename(item["audio"])
  audio_path = os.path.join(audio_path, filename)
  reference = item["text"]

  try:
      result = asr_pipeline(audio_path, return_timestamps = True)
      hypothesis = result["text"]
  except Exception as e:
      print(f"Skipping {audio_path}: {e}")
      continue

  ground_truth.append(reference.lower())
  predictions.append(hypothesis.lower())

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeSta

### Normalize to enable JIWER processing

In [ ]:
transform = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.RemovePunctuation(),
    jiwer.RemoveMultipleSpaces(),
    jiwer.Strip()
])